In [33]:
# Config
import os, glob, json, math
import numpy as np
import pandas as pd

# ---- Paths ----
BASE_DIR      = "../data/refs"
PTFE_DIR      = os.path.join(BASE_DIR, "ptfe_1000")  # 1000 CSV for pure PTFE
HBN_DIR       = os.path.join(BASE_DIR, "hbn_thin_1000")   # 1000 CSV for pure hBN
OUT_DIR       = "../generated/bulk_50k"               # OUTPUT FOLDER (NPZ shards)
os.makedirs(OUT_DIR, exist_ok=True)

# Film physics 
t_ptfe_um = 200.0    # PTFE film thickness (µm) 
t_hbn_um  = 540.0   # hBN "reference" thickness (µm)  (updated for the thin pellet)
rho_ptfe  = 2.20     # g/cm^3
rho_hbn   = 0.3     # g/cm^3  (updated for the thin pellet)

# When mixing, we “record” on a PTFE-like film of this thickness:
t_mix_um  = t_ptfe_um

# ---- Dataset settings ----
N_SYNTH          = 50_000           # total synthetic samples to generate
HBN_MIN, HBN_MAX = 0, 50            # inclusive range for random hBN %
RANDOM_SEED      = 42               # reproducibility
np.random.seed(RANDOM_SEED)

# ---- Saving (shards) ----
SHARD_SIZE = 5_000                  # 50k -> 10 shards of 5k; adjust if needed
assert N_SYNTH % SHARD_SIZE == 0, "For simplicity, set N_SYNTH multiple of SHARD_SIZE"
META_JSON = os.path.join(OUT_DIR, "meta.json")

print("Config ready.")


Config ready.


In [34]:
# Utility functions
from typing import List, Tuple

def build_file_list(folder: str) -> List[str]:
    files = sorted(glob.glob(os.path.join(folder, "*.CSV")))
    if not files:
        files = sorted(glob.glob(os.path.join(folder, "*.csv")))
    if not files:
        raise FileNotFoundError(f"No CSV files found in: {folder}")
    return files

def load_xy_any(fn: str) -> Tuple[np.ndarray, np.ndarray]:
    """Load 2-col CSV (header/no-header tolerant) -> (x, y) as float arrays."""
    try:
        df = pd.read_csv(fn, header=0)
        x = pd.to_numeric(df.iloc[:,0], errors="coerce").to_numpy()
        y = pd.to_numeric(df.iloc[:,1], errors="coerce").to_numpy()
        if np.isnan(x).any() or np.isnan(y).any(): raise ValueError
    except Exception:
        df = pd.read_csv(fn, header=None)
        x = pd.to_numeric(df.iloc[:,0], errors="coerce").to_numpy()
        y = pd.to_numeric(df.iloc[:,1], errors="coerce").to_numpy()
        mask = ~(np.isnan(x) | np.isnan(y))
        x, y = x[mask], y[mask]
    return x, y

print("Utils ready.")


Utils ready.


In [35]:
# Load raw, find common wn grid, interp
ptfe_files = build_file_list(PTFE_DIR)
hbn_files  = build_file_list(HBN_DIR)
print(f"Found PTFE: {len(ptfe_files)} files | hBN: {len(hbn_files)} files")

# Load all x,y once to find overlap
raw_ptfe = [load_xy_any(f) for f in ptfe_files]
raw_hbn  = [load_xy_any(f) for f in hbn_files]

# Determine safe/common wavenumber window
wn_min = max(min(x.min() for x,_ in raw_ptfe),
             min(x.min() for x,_ in raw_hbn))
wn_max = min(max(x.max() for x,_ in raw_ptfe),
             max(x.max() for x,_ in raw_hbn))

assert wn_max - wn_min > 500, "Common spectral window too small; check raw files."

# 1 cm^-1 spacing
wn = np.arange(math.ceil(wn_min), math.floor(wn_max)+1, 1.0)

def make_bank(raw_list, file_list):
    bank = []
    for (x,y), fn in zip(raw_list, file_list):
        yi = np.interp(wn, x, y)
        bank.append({"file": os.path.basename(fn), "A_raw": yi})
    return bank

ptfe_bank = make_bank(raw_ptfe, ptfe_files)
hbn_bank  = make_bank(raw_hbn,  hbn_files)
print("Interpolated to common grid; wn points:", len(wn))


Found PTFE: 1000 files | hBN: 1000 files
Interpolated to common grid; wn points: 3323


In [36]:
# LSQ-scale to canonical medians; areal-mass norm

def lsq_scale_to_ref(A, ref):
    """Return scalar s that minimizes ||s*A - ref||_2."""
    denom = float(np.dot(A, A))
    return float(np.dot(A, ref) / denom) if denom > 0 else 1.0

# Build initial group medians
ptfe_stack = np.vstack([d["A_raw"] for d in ptfe_bank])
hbn_stack  = np.vstack([d["A_raw"] for d in hbn_bank])
ptfe_med   = np.median(ptfe_stack, axis=0)
hbn_med    = np.median(hbn_stack,  axis=0)

# Scale each to median
for d in ptfe_bank:
    s = lsq_scale_to_ref(d["A_raw"], ptfe_med)
    d["scale_to_med"] = s
    d["A_scaled"]     = s * d["A_raw"]

for d in hbn_bank:
    s = lsq_scale_to_ref(d["A_raw"], hbn_med)
    d["scale_to_med"] = s
    d["A_scaled"]     = s * d["A_raw"]

# Recompute canonical medians after scaling (optional refinement)
ptfe_stack2 = np.vstack([d["A_scaled"] for d in ptfe_bank])
hbn_stack2  = np.vstack([d["A_scaled"] for d in hbn_bank])
ptfe_ref    = np.median(ptfe_stack2, axis=0)
hbn_ref     = np.median(hbn_stack2,  axis=0)

# Areal-mass normalization
um_to_cm  = 1e-4
t_ptfe_cm = t_ptfe_um * um_to_cm
t_hbn_cm  = t_hbn_um  * um_to_cm
t_mix_cm  = t_mix_um  * um_to_cm

def to_areal_mass_norm(A_scaled, rho, t_cm):
    return A_scaled / (rho * t_cm)

for d in ptfe_bank:
    d["A_norm"] = to_areal_mass_norm(d["A_scaled"], rho_ptfe, t_ptfe_cm)

for d in hbn_bank:
    d["A_norm"] = to_areal_mass_norm(d["A_scaled"], rho_hbn, t_hbn_cm)

print("Bank harmonized & normalized.")


Bank harmonized & normalized.


In [37]:
# Mixing helper (PTFE + hBN, %)
def mix_spectrum(ptfe_percent, A_ptfe_norm, A_hbn_norm, rho_ptfe, rho_hbn, t_mix_cm):
    """
    Returns A_out (absorbance on film t_mix_cm) and rho_mix.
    ptfe_percent in [0,100]; hBN% = 100 - PTFE%.
    """
    w_hbn  = (100.0 - ptfe_percent) / 100.0
    w_ptfe = 1.0 - w_hbn
    A_mix_norm = w_ptfe * A_ptfe_norm + w_hbn * A_hbn_norm
    rho_mix = 1.0 / (w_ptfe / rho_ptfe + w_hbn / rho_hbn)
    A_out   = A_mix_norm * (rho_mix * t_mix_cm)
    return A_out, float(rho_mix)

print("Mix helper ready.")


Mix helper ready.


In [38]:
# Synthetic sampler (random pairs & ratios)
N_PTFE = len(ptfe_bank)
N_HBN  = len(hbn_bank)
assert N_PTFE >= 1000 and N_HBN >= 1000, "Expecting ~1000 CSVs each per your new setup."

# Pre-extract normalized arrays for speed
PTFE_NORM = np.vstack([d["A_norm"] for d in ptfe_bank])  # shape: (N_PTFE, n_wn)
HBN_NORM  = np.vstack([d["A_norm"]  for d in hbn_bank])  # shape: (N_HBN , n_wn)

def sample_batch(batch_size: int, hbn_min=0, hbn_max=50):
    """
    Returns:
      X : (batch_size, n_wn) spectra
      y_hbn_pct : (batch_size,) integer hBN %
      y_ptfe_pct: (batch_size,) integer PTFE %
    """
    # Randomly choose source indices (with replacement for diversity)
    ptfe_idx = np.random.randint(0, N_PTFE, size=batch_size)
    hbn_idx  = np.random.randint(0, N_HBN,  size=batch_size)

    # Random hBN% in [hbn_min, hbn_max]
    y_hbn_pct = np.random.uniform(hbn_min, hbn_max, size=batch_size).astype(np.float32)
    y_ptfe_pct = (100 - y_hbn_pct).astype(np.int16)

    # Gather source spectra
    A_ptfe = PTFE_NORM[ptfe_idx]   # (B, n_wn)
    A_hbn  = HBN_NORM[hbn_idx]     # (B, n_wn)

    # Mix per-sample
    w_hbn  = (y_hbn_pct / 100.0).reshape(-1, 1)
    w_ptfe = 1.0 - w_hbn

    # rho_mix per sample
    rho_mix = 1.0 / (w_ptfe / rho_ptfe + w_hbn / rho_hbn)  # (B,1)
    A_mix_norm = w_ptfe * A_ptfe + w_hbn * A_hbn           # (B, n_wn)
    X = A_mix_norm * (rho_mix * t_mix_cm)                  # (B, n_wn)

    return X.astype(np.float32), y_hbn_pct, y_ptfe_pct


In [39]:
# Generate N_SYNTH samples and save as compressed NPZ
n_points = len(wn)
n_shards = N_SYNTH // SHARD_SIZE

# Save small metadata json
meta = {
    "description": "Synthetic IR mixtures for MLP training (PTFE+hBN)",
    "n_samples": N_SYNTH,
    "n_shards": n_shards,
    "shard_size": SHARD_SIZE,
    "wavenumber_min": float(wn.min()),
    "wavenumber_max": float(wn.max()),
    "wavenumber_step": float(wn[1]-wn[0]) if len(wn) > 1 else None,
    "n_points": int(n_points),
    "t_ptfe_um": float(t_ptfe_um),
    "t_hbn_um": float(t_hbn_um),
    "t_mix_um": float(t_mix_um),
    "rho_ptfe": float(rho_ptfe),
    "rho_hbn": float(rho_hbn),
    "hbn_percent_range": [int(HBN_MIN), int(HBN_MAX)],
    "random_seed": int(RANDOM_SEED),
}
with open(META_JSON, "w") as f:
    json.dump(meta, f, indent=2)
print(f"Saved meta → {META_JSON}")

# Create shards
for s in range(n_shards):
    X, y_hbn, y_ptfe = sample_batch(SHARD_SIZE, HBN_MIN, HBN_MAX)
    shard_path = os.path.join(OUT_DIR, f"shard_{s:03d}.npz")
    # Use numpy's compressed format
    np.savez_compressed(
        shard_path,
        X=X,                 # float32, shape (SHARD_SIZE, n_points)
        y_hbn=y_hbn,         # int16,   shape (SHARD_SIZE,)
        y_ptfe=y_ptfe,       # int16,   shape (SHARD_SIZE,)
        wn=wn.astype(np.float32)  # store axis once per shard for convenience
    )
    print(f"[{s+1}/{n_shards}] Saved:", shard_path)

print("All shards saved to:", OUT_DIR)


Saved meta → ../generated/bulk_50k/meta.json
[1/10] Saved: ../generated/bulk_50k/shard_000.npz
[2/10] Saved: ../generated/bulk_50k/shard_001.npz
[3/10] Saved: ../generated/bulk_50k/shard_002.npz
[4/10] Saved: ../generated/bulk_50k/shard_003.npz
[5/10] Saved: ../generated/bulk_50k/shard_004.npz
[6/10] Saved: ../generated/bulk_50k/shard_005.npz
[7/10] Saved: ../generated/bulk_50k/shard_006.npz
[8/10] Saved: ../generated/bulk_50k/shard_007.npz
[9/10] Saved: ../generated/bulk_50k/shard_008.npz
[10/10] Saved: ../generated/bulk_50k/shard_009.npz
All shards saved to: ../generated/bulk_50k


In [40]:
# Quick sanity check: load shard 0
test_npz = os.path.join(OUT_DIR, "shard_000.npz")
dat = np.load(test_npz)
print("Keys:", list(dat.keys()))
print("Shapes:", dat["X"].shape, dat["y_hbn"].shape, dat["wn"].shape)
print("Example hBN%:", dat["y_hbn"][:10])


Keys: ['X', 'y_hbn', 'y_ptfe', 'wn']
Shapes: (5000, 3323) (5000,) (3323,)
Example hBN%: [10.073196  13.2604265  8.48796    4.782035  20.956808   5.6649804
  8.595628  27.366234   8.229963   3.1939883]
